# Assignment Problem: Optimization for Cost-Efficient Resource Allocation

## Context

The Assignment Problem is a fundamental optimization model in Operations Research
used to allocate limited resources to competing tasks in an optimal manner.
Each resource can be assigned to at most one task, and each task must be assigned
to exactly one resource.

This problem arises in a wide range of real-world applications such as workforce
assignment, machine-job allocation, task scheduling, and service dispatching.
The model focuses on making discrete (yes/no) assignment decisions while minimizing
total operational cost.

In this notebook, the classical **Assignment Problem** is formulated as a
**Mixed-Integer Linear Program (MILP)** and solved using both open-source and
commercial optimization solvers.

**Solvers used:**
- CBC (open-source)
- IBM CPLEX Community Edition

To highlight the role of integrality, the notebook also examines the **LP relaxation**
of the model and explains why binary constraints are essential for obtaining
meaningful assignment decisions.


## Business Problem Statement

An organization must assign a set of workers to a set of tasks.
Each worker can perform at most one task, and each task requires exactly one worker.
Assigning a particular worker to a particular task incurs a known cost that reflects
factors such as skill mismatch, time, or operational expense.

The **Goal** is to determine the optimal assignment of workers to tasks such that:
- Every task is assigned to exactly one worker,
- No worker is assigned to more than one task,
- The total assignment cost is minimized.


## Why This Problem Matters

The Assignment Problem appears across a wide range of operational and
decision-making contexts where one-to-one matching is required.
Typical real-world applications include assigning workers to jobs,
machines to tasks, drivers to delivery routes, reviewers to submitted papers,
and technicians to service requests.

As a foundational binary optimization model, the assignment problem forms
the basis for more advanced formulations such as workforce scheduling,
routing, and resource planning.


## Mathematical Formulation

#### Sets and Parameters
\begin{align}
i&\in\{1,2,\ldots,N\}, \text{ set of workers}\\
j&\in\{1,2,\ldots,N\}, \text{ set of tasks}\\
c_{ij} &: \text{Cost of assigning worker }i \text{ to task }j
\end{align}

#### Decision Variables
\begin{equation}
x_{ij}:\begin{cases}
1 & \text{ if worker }i \text{ is assigned to task }j\\
0 & \text{otherwise}
\end{cases}
\end{equation}

#### Objective Function
\begin{equation}
\min \enspace \sum_{i=1}^N\sum_{j=1}^N c_{ij}~x_{ij}
\end{equation}
**Interpretation:** By summing over all workers and tasks, the objective aggregates the costs of all chosen assignments and selects the combination that yields the **minimum overall cost** while satisfying the assignment constraints.

#### Constraints
1. **Worker Assignment Constraint** \begin{equation} \sum_{j=1}^N x_{ij}\leq1,\enspace\forall~i \end{equation} **Interpretation:** Each worker is assigned to almost one task.
2. **Task Assignment Constraint** \begin{equation}\sum_{i=1}^Nx_{ij}=1,\enspace\forall~j\end{equation} **Interpretation:** Each task is assigned exactly to one worker
3. **Binary Constraints** \begin{equation}x_{ij}\in\{0,1\},\enspace\forall~i,\forall~j.\end{equation}

## Data and Modeling Setup

A small illustrative dataset is used to demonstrate the assignment model.
The dataset consists of a set of workers, a set of tasks, and a cost
associated with assigning each worker to a task. This example is intentionally kept small to allow easy verification
and interpretation of the optimal assignment.


In [26]:
import pyomo.environ as pyo

workers = ['W1', 'W2', 'W3']
tasks = ['T1', 'T2', 'T3']

cost = {
    ('W1','T1'):13, ('W1','T2'):7, ('W1','T3'):9,
    ('W2','T1'):8, ('W2','T2'):7, ('W2','T3'):4,
    ('W3','T1'):6, ('W3','T2'):12, ('W3','T3'):10
}

## Pyomo Modeling and Solver Execution

In this section, the assignment problem is implemented using Pyomo.
The mathematical formulation introduced earlier is translated directly
into a binary optimization model and solved using a mixed-integer solver.


### Model Formulation in Pyomo

The assignment problem is formulated as a binary optimization model.
Binary decision variables indicate whether a specific worker is
assigned to a specific task.

The model enforces:
- Exactly one worker assigned to each task
- At most one task is assigned to each worker.


In [27]:
# Building the MILP optimation model in Pyomo

def build_assignment_problem(workers, tasks, cost):
    model = pyo.ConcreteModel()

    # Define the sets
    model.W = pyo.Set(initialize = workers)
    model.T = pyo.Set(initialize = tasks)

    # Declare the decision variables
    model.x = pyo.Var(model.W, model.T, domain = pyo.Binary)

    # Define the Objective function
    def obj_function(m):
        return sum(cost[i,j] * m.x[i,j] for i in m.W for j in m.T)
    model.obj = pyo.Objective(expr = obj_function, sense = pyo.minimize)

    # Adding constraints of the optimization problem
    # Constraint 1: Worker assignment constraint
    def worker_assgn_rule(m,i):
        return sum(m.x[i,j] for j in m.T) <= 1
    model.worker_assignment = pyo.Constraint(model.W, rule = worker_assgn_rule)

    # Constraint 2: Task assignment constraint
    def task_assgn_rule(m,j):
        return sum(m.x[i,j] for i in m.W) == 1
    model.task_assignment = pyo.Constraint(model.T, rule=task_assgn_rule)

    return model

    

### Reusable Solver Execution

To ensure solver-agnostic modeling and improve code reusability, a general
solver execution function is defined. This function takes a Pyomo model
and a solver name as inputs and returns the solver results.
This design allows the same optimization model to be solved using different
solvers without modifying the model formulation.


In [28]:
# Solving the optimization problem with the given solver

def solve_assignment_problem(model, solver_name):
    solver = pyo.SolverFactory(solver_name)
    results = solver.solve(model, tee = False)
    return results

### Solution Extraction and Printing

After solving the optimization model, the solution needs to be interpreted
in a structured and readable form.
A solution displaying function is defined to extract the active
assignment decisions and present them in a clear tabular format, along with
the optimal cost value.


In [29]:
import pandas as pd

# to display the optimal assignments

def print_worker_task_assignment(model, title = "Worker to Task Assignment"):
    assignment_schedule = []

    for i in model.W:
        for j in model.T:
            if pyo.value(model.x[i,j]) > 0.5:
                assignment_schedule.append(
                    {
                        "Worker":i,
                        "Task" : j
                    }
                )
    line = "="*len(title)    
    print(f"\n{line}\n{title}\n{line}")
    
    assignment_schedule_df = pd.DataFrame(assignment_schedule)
    optimal_cost = pyo.value(model.obj)

    return assignment_schedule_df

### Solving the Illustrative Example

The assignment model is now solved using the reusable solver interfaces.
The extracted solution highlights the optimal worker–task assignments
and the minimum total cost of assignment.


In [30]:
# Step 1: Building the optimization model in Pyomo
model = build_assignment_problem(workers, tasks, cost)

# Step 2: Decide the solver to solve the optimization problem
solver_name = 'cbc'

# Step 3: Solve the optimization problem with the given solver
results = solve_assignment_problem(model, solver_name)

# Step 4: Printing the optimal value of the objective function
optimal_cost = pyo.value(model.obj)
print(f"Optimal Assignment Cost = {optimal_cost}")

# Step 5: Printing the worker-task assignment table
print_worker_task_assignment(model, title = "Worker to Task Assignment")


Optimal Assignment Cost = 17.0

Worker to Task Assignment


,Worker,Task
0,W1,T2
1,W2,T3
2,W3,T1


### Interpretation

The model successfully assigns each task to exactly one worker while
ensuring that no worker is assigned to more than one task.
The resulting assignment minimizes the total cost among all feasible
worker-task pairs for the given dataset.


## Solver Comparison

To demonstrate solver portability and validate the robustness of the
assignment problem formulation, the same model is solved using CPLEX (Community Edition), an
alternative mixed-integer solver and the results obtained are compared
against the CBC baseline.


#### CPLEX Availability Check

Before solving the model using CPLEX, solver availability is verified.


In [31]:
pyo.SolverFactory('cplex').available()

True

In [32]:
model_cplex = build_assignment_problem(workers, tasks, cost)
solver_name = 'cplex'
results_cplex = solve_assignment_problem(model_cplex,solver_name)
cplex_optimal_cost = pyo.value(model_cplex.obj)
print(f"Optimal Assignment Cost = {cplex_optimal_cost}")
print_worker_task_assignment(model_cplex, title = "Worker to Task Assignment")


Optimal Assignment Cost = 17.0

Worker to Task Assignment


,Worker,Task
0,W1,T2
1,W2,T3
2,W3,T1


### Solver Comparison Results

The table below summarizes the optimal objective values obtained using
different solvers for the same assignment problem example.


In [33]:
import pandas as pd

solver_comparison = pd.DataFrame({
    "Solver": ["CBC", "CPLEX (CE)"],
    "Optimal Cost": [optimal_cost, cplex_optimal_cost]
})

solver_comparison.style.hide(axis="index").format({"Optimal Cost": "{:.2f}"})


Solver,Optimal Cost
CBC,17.00
CPLEX (CE),17.00


### Interpretation of Solver Comparison

Both CBC and CPLEX return the same optimal objective value for the
assignment problem, confirming the correctness of the formulation and
its independence from solver choice.
This demonstrates that the binary optimization model is solver-agnostic
and can be reliably solved using both open-source and commercial
mixed-integer programming solvers.


## LP Relaxation of the Assignment Problem

To highlight the role of integrality in assignment decisions, the model
is also examined under its linear programming (LP) relaxation. In the
LP relaxation, the binary decision variables are replaced by continuous
variables bounded between 0 and 1.

In [34]:
def build_assignment_problem_lp_relaxation(workers, tasks, cost):
    model = pyo.ConcreteModel()

    model.W = pyo.Set(initialize = workers)
    model.T = pyo.Set(initialize = tasks)

    model.x = pyo.Var(model.W, model.T, domain = pyo.UnitInterval)

    model.obj = pyo.Objective(
        expr = sum(model.x[i,j]*cost[i,j] for i in model.W for j in model.T),
        sense = pyo.minimize
    )

    def worker_assgn_rule(model, i):
        return sum(model.x[i,j] for j in model.T) <= 1

    def task_assgn_rule(model, j):
        return sum(model.x[i,j] for i in model.W) == 1
    
    model.worker_assignment = pyo.Constraint(model.W, rule = worker_assgn_rule)
    model.task_assignment = pyo.Constraint(model.T, rule = task_assgn_rule)

    return model

In [35]:
model_lp = build_assignment_problem_lp_relaxation(workers, tasks, cost)

solverName = 'cbc'
results_lp = solve_assignment_problem(model_lp, solverName)
lp_optimal_cost = pyo.value(model_lp.obj)
print(f"Optimal cost with the LP relaxation = {lp_optimal_cost}")
print_worker_task_assignment(model_cplex, title = "Worker to Task Assignment")

Optimal cost with the LP relaxation = 17.0

Worker to Task Assignment


,Worker,Task
0,W1,T2
1,W2,T3
2,W3,T1


### Interpretation of the LP Relaxation

In the LP relaxation, the binary assignment variables are allowed to take
fractional values between 0 and 1. While the relaxed model may achieve
an objective value that is equal to or lower than the integer solution,
the resulting assignments can be fractional and therefore lack a clear
operational interpretation.

For example, a task may be partially assigned to multiple workers,
which is not meaningful in practical assignment settings. This
demonstrates that integrality constraints are essential for enforcing
discrete, one-to-one assignment decisions.

Although the classical assignment problem can yield integer solutions
under an LP relaxation, the binary formulation is used here to clearly
represent discrete assignment decisions and to remain consistent with
more general scheduling and allocation models.


## Large Scale Assignment Scenario

To demonstrate that the assignment model is not restricted to a small
illustrative example, a larger assignment scenario is generated and
solved using the same formulation. This highlights the generality and
reusability of the proposed optimization model.


In [36]:
import random
def generate_assignment_dataset(num_workers, num_tasks, seed=42):
    random.seed(seed)

    workers = [f"W{i}" for i in range(1, num_workers+1)]
    tasks = [f"T{j}" for j in range(1, num_tasks+1)]

    cost = {
        (i, j): random.randint(1,100) for i in workers for j in tasks
    }

    return workers, tasks, cost

In [37]:
num_workers = 50
num_tasks = 50
workers_ls, tasks_ls, cost_ls = generate_assignment_dataset(num_workers, num_tasks)

solver_name = 'cbc'
model_ls = build_assignment_problem(workers_ls, tasks_ls, cost_ls)
results_ls = solve_assignment_problem(model_ls, solver_name)
optimal_ls = pyo.value(model_ls.obj)
print(f"Optimal cost = {optimal_ls}")
# print_worker_task_assignment(model_ls, title = "Worker to Task Assignment")


Optimal cost = 217.0


### Interpretation of the large scale scenario
The assignment model scales naturally to large-scale assignment problem using
the same mathematical formulation and implementation structure. By
generating a larger cost matrix and reusing the existing
modeling components, the problem can be solved without any changes to
the decision variables, constraints, or objective function.

This highlights the generality of the formulation and the modular design
of the Pyomo implementation, which allows the model to be extended to
more realistic problem sizes in a straightforward and reproducible
manner.


## Key Takeaways 

- The assignment problem was formulated as a binary optimization model
  that enforces one-to-one matching between workers and tasks.

- The same formulation was solved using both CBC and CPLEX solvers,
  yielding identical optimal objective values and confirming solver
  independence of the model.

- An LP relaxation of the model was examined to highlight the importance
  of integrality constraints in producing meaningful, discrete assignment
  decisions.

- A large-scale assignment scenario was solved using the same
  formulation, demonstrating that the model scales naturally to more
  realistic problem sizes without requiring changes to the underlying
  implementation.

Overall, this project illustrates how a clean mathematical formulation
and modular optimization implementation can support correctness,
solver portability, and scalability in assignment-type decision problems.
